[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/xarss/ProjetoTransformador1/blob/main/ProjetoTransformador1.ipynb)

# Download all necessary libraries

In [ ]:
import os
import requests

if(not os.path.exists("requirements.txt")):
  print("Downloading Requirements")

  requirements_url = "https://github.com/xarss/ProjetoTransformador1/raw/main/requirements.txt"
  requirements_path = "requirements.txt"

  response = requests.get(requirements_url)
  print(response.content)
  with open(requirements_path, 'wb') as file:
    file.write(response.content)

  !pip install -r requirements.txt

# Imports

In [ ]:
import os
import git
import zipfile
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Constants

In [ ]:
OUTPUT_DIR = 'Data'
IMAGE_DIR = os.path.join(OUTPUT_DIR, 'Images')
REPO_URL = "https://github.com/xarss/ProjetoTransformador1.git"
REPO_DIR = "ProjetoTransformador1"

# Clone repo to get features and images that are saved in LFS

In [ ]:
if not os.path.exists(REPO_DIR):
    print("Cloning the repository...")
    git.Repo.clone_from(REPO_URL, REPO_DIR)
    print("Repository cloned.")

os.makedirs(OUTPUT_DIR, exist_ok=True)

os.chdir(REPO_DIR)

!git lfs pull

Cloning the repository...
Repository cloned.


# Create features Data Frame and unzip images

In [ ]:
for csv_file in ['features_mfcc.csv', 'features_lfcc.csv', 'features_logmel.csv']:
    source_path = os.path.join(REPO_DIR, 'Data', csv_file)
    target_path = os.path.join(OUTPUT_DIR, csv_file)
    if not os.path.exists(target_path):
        print(f"Copying {csv_file}...")
        os.rename(source_path, target_path)
        print(f"Copied {csv_file}.")

zip_source = os.path.join(REPO_DIR, 'Data', 'images.zip')
zip_target = os.path.join(OUTPUT_DIR, 'images.zip')
if not os.path.exists(zip_target):
    print("Copying images.zip...")
    os.rename(zip_source, zip_target)
    print("Copied images.zip.")

# Step 6: Unzip images
if not os.path.exists(IMAGE_DIR):
    os.makedirs(IMAGE_DIR, exist_ok=True)

with zipfile.ZipFile(zip_target, 'r') as zip_ref:
    zip_ref.extractall(IMAGE_DIR)
    print("Images unzipped.")

mfcc_df = pd.read_csv(os.path.join(OUTPUT_DIR, 'features_mfcc.csv'))
lfcc_df = pd.read_csv(os.path.join(OUTPUT_DIR, 'features_lfcc.csv'))
logmel_df = pd.read_csv(os.path.join(OUTPUT_DIR, 'features_logmel.csv'))

features_df = pd.merge(mfcc_df, lfcc_df, on='file_name', suffixes=('_mfcc', '_lfcc'))
features_df = pd.merge(features_df, logmel_df, on='file_name')

features_df.head()

Images unzipped.


,file_name,label_mfcc,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,...,logmel_119,logmel_120,logmel_121,logmel_122,logmel_123,logmel_124,logmel_125,logmel_126,logmel_127,logmel_128
0,DF_E_2000011,0,-258.15650,68.461950,-4.070857,41.815220,-12.947648,0.652590,-32.048534,-10.525288,...,-38.371040,-38.522770,-39.003820,-38.747696,-37.935814,-37.926888,-39.174030,-40.045155,-42.009910,-43.053830
1,DF_E_2000013,0,-197.49075,74.101060,-20.719416,21.303389,3.568402,-23.902008,-21.944940,-15.401686,...,-28.880777,-28.182413,-28.006674,-27.734747,-27.468850,-28.263678,-30.419197,-32.445484,-33.184155,-33.075750
2,DF_E_2000024,0,-245.88821,58.375427,3.938906,27.994371,-4.515254,-2.728295,-31.741747,-4.433678,...,-27.978386,-28.901148,-29.482600,-30.001705,-27.182116,-24.936586,-28.185112,-47.615543,-50.954940,-50.954940
3,DF_E_2000026,0,-258.21800,64.792450,-26.174234,31.422873,-39.746807,-4.368946,-24.203897,-17.594040,...,-38.589290,-38.543840,-39.140380,-39.473120,-39.367550,-39.460510,-41.005024,-47.146620,-47.207413,-47.225680
4,DF_E_2000027,0,-285.02810,91.180786,-16.007261,40.066260,-11.105558,-14.232035,-10.550928,-19.979368,...,-39.566300,-40.229347,-40.474980,-41.828495,-42.607613,-42.541990,-44.043407,-48.737200,-48.762028,-48.762028


In [ ]:
X = features_df.drop(columns=['file_name', 'label', 'label_mfcc', 'label_lfcc'])
y = features_df['label']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
svm_preds = svm.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_preds)
print(f"SVM Accuracy: {svm_accuracy * 100:.2f}%")

SVM Accuracy: 94.67%


In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_preds)
print(f"Random Forest Accuracy: {rf_accuracy * 100:.2f}%")

Random Forest Accuracy: 96.67%


In [ ]:
def load_image(file_path, img_size=(128, 128)):
    img = image.load_img(file_path, target_size=img_size, color_mode='rgb')
    img_array = image.img_to_array(img) / 255.0
    return img_array

image_files = []
y_img = []

for feature_type in ['mfcc', 'lfcc', 'logmel']:
    for _, row in features_df.iterrows():
        file_name = row['file_name'] + ".png"
        image_path = os.path.join(IMAGE_DIR, feature_type, file_name)

        image_files.append(image_path)

        y_img.append(int(row['label']))

print(f"Found {len(image_files)} image files.")

X_img = np.array([load_image(file) for file in image_files])

X_train_img, X_temp_img, y_train_img, y_temp_img = train_test_split(X_img, y_img, test_size=0.3, random_state=42)
X_val_img, X_test_img, y_val_img, y_test_img = train_test_split(X_temp_img, y_temp_img, test_size=0.5, random_state=42)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_img, y_train_img, validation_data=(X_val_img, y_val_img), epochs=5, batch_size=32)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test_img, y_test_img)
print(f"Shallow CNN Test Accuracy: {test_acc * 100:.2f}%")

Found 6000 image files.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Unrecognized data type: x=[[[[0.         0.         0.01568628]
   [0.         0.         0.01568628]
   [0.         0.         0.01568628]
   ...
   [0.         0.         0.01568628]
   [0.         0.         0.01568628]
   [0.         0.         0.01568628]]

  [[0.         0.         0.01568628]
   [0.         0.         0.01568628]
   [0.         0.         0.01568628]
   ...
   [0.         0.         0.01568628]
   [0.         0.         0.01568628]
   [0.         0.         0.01568628]]

  [[0.         0.         0.01568628]
   [0.         0.         0.01568628]
   [0.         0.         0.01568628]
   ...
   [0.         0.         0.01568628]
   [0.         0.         0.01568628]
   [0.         0.         0.01568628]]

  ...

  [[0.07843138 0.05490196 0.21176471]
   [0.13333334 0.06666667 0.3137255 ]
   [0.23921569 0.05882353 0.44313726]
   ...
   [0.00392157 0.         0.01960784]
   [0.         0.         0.01568628]
   [0.         0.         0.01568628]]

  [[0. 